In [1]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        fill_mode='nearest',  # set mode for filling points outside the input boundaries
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        rescale=None,  # set rescaling factor (applied before any other transformation)
        preprocessing_function=None,  # set function that will be applied on each input
        data_format=None,  # image data format, either "channels_first" or "channels_last"
        validation_split=0.0)  # fraction of images reserved for validation (strictly between 0 and 1)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Using TensorFlow backend.


170500096/170498071 [==============================] - 367s 2us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 154s 99ms/step - loss: 1.8545 - acc: 0.3159 - val_loss: 1.6336 - val_acc: 0.4071
Epoch 2/100
1563/1563 [==============================] - 156s 100ms/step - loss: 1.5732 - acc: 0.4229 - val_loss: 1.3649 - val_acc: 0.5074
Epoch 3/100
1563/1563 [==============================] - 158s 101ms/step - loss: 1.4627 - acc: 0.4674 - val_loss: 1.3074 - val_acc: 0.5270
Epoch 4/100
1563/1563 [==============================] - 144s 92ms/step - loss: 1.3800 - acc: 0.5046 - val_loss: 1.2586 - val_acc: 0.5531
Epoch 5/100
1563/1563 [==============================] - 141s 90ms/step - loss: 1.3136 - acc: 0.5285 - val_loss: 1.1397 - val_acc: 0.5896
Epoch 6/100
1563/1563 [==============================] - 142s 91ms/step - loss: 1.2635 - acc: 0.5485 - val_loss: 1.1373 - val_acc: 

1563/1563 [==============================] - 139s 89ms/step - loss: 0.7980 - acc: 0.7319 - val_loss: 0.7214 - val_acc: 0.7569
Epoch 59/100
1563/1563 [==============================] - 140s 90ms/step - loss: 0.7946 - acc: 0.7330 - val_loss: 0.7552 - val_acc: 0.7461
Epoch 60/100
1563/1563 [==============================] - 141s 90ms/step - loss: 0.7913 - acc: 0.7352 - val_loss: 0.7442 - val_acc: 0.7506
Epoch 61/100
1563/1563 [==============================] - 140s 90ms/step - loss: 0.7942 - acc: 0.7347 - val_loss: 0.7294 - val_acc: 0.7616
Epoch 62/100
1563/1563 [==============================] - 140s 90ms/step - loss: 0.7986 - acc: 0.7319 - val_loss: 0.7035 - val_acc: 0.7586
Epoch 63/100
1563/1563 [==============================] - 139s 89ms/step - loss: 0.7995 - acc: 0.7312 - val_loss: 0.7017 - val_acc: 0.7670
Epoch 64/100
1563/1563 [==============================] - 141s 90ms/step - loss: 0.7911 - acc: 0.7351 - val_loss: 0.7069 - val_acc: 0.7617
Epoch 65/100
1563/1563 [================

In [2]:
from keras.models import load_model

model.save('cifar10_cnn_model.h5') # creates a HDF5 file 'my_model.h5'
with open('cifar10_cnn_model.json', 'w') as fout:
    fout.write(model.to_json())